In [1]:
import sqlite3
import time

import igraph

In [2]:
db = 'edge.db'
conn = sqlite3.connect(f'file:{db}?mode=ro', uri=True)
cur = conn.cursor()

def get_edge_count():
    cur.execute('''SELECT COUNT(*) FROM Edge;''')
    return cur.fetchone()[0]

def get_edge_cursor():
    cur.execute('''SELECT src, dst, weight FROM Edge;''')
    return cur

In [3]:
stime = time.time()
g = igraph.Graph.TupleList(get_edge_cursor(), weights=True)
etime = time.time()
print(f'Construct graph done during {etime-stime}')

Construct graph done during 7.80655574798584


In [4]:
stime = time.time()
g.write_pickle('graph.igraph')
etime = time.time()
print(f'Graph Pickling Done Done during {etime-stime}')

Graph Pickling Done Done during 6.5604870319366455


In [ ]:
conn.close()